In [1]:
import numpy as np

import pandas as pd
from pandas.api.types import CategoricalDtype

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model as lm
from sklearn import preprocessing as pre
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import quandl
quandl.ApiConfig.api_key = "tzt74qzzscPX2KqxS_rD"

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

In [2]:
# Load info for sp500
sp500_general = pd.read_csv('constituents.csv')
sp500_risks = pd.read_csv('company_risk.csv')
sp500 = sp500_general.merge(sp500_risks, on='Symbol')

# Normalize the TotalRisk Column.
min_max_scaler = pre.MinMaxScaler()
sp500['Risk'] = min_max_scaler.fit_transform(sp500['Risk'].values.reshape(-1, 1)).flatten()

sp500.head()

,Symbol,Name,Sector,Risk
0,MMM,3M Company,Industrials,0.000000
1,ABT,Abbott Laboratories,Health Care,0.290338
2,ABBV,AbbVie,Health Care,0.374926
3,ACN,Accenture plc,Information Technology,0.258289
4,ATVI,Activision Blizzard,Information Technology,0.395041


In [3]:
data = quandl.get_table('WIKI/PRICES',
                        qopts = { 'columns': ['ticker', 'date', 'close'] },
                        ticker = ['AAPL', 'MSFT', 'FB'],
                        date = { 'gte': '2013-01-01', 'lte': '2018-03-30' },
                        paginate=True)
data['ticker'].value_counts()

FB      1317
MSFT    1317
AAPL    1316
Name: ticker, dtype: int64

In [4]:
data['ticker'] = data['ticker'].astype('category').cat.codes
data['day'] = (data['date'] - data['date'].min()).dt.days
data = data.drop(['date'], axis=1)
data['last_day_close'] = data.groupby(['ticker'])['close'].shift()
data['last_day_diff'] = data.groupby(['ticker'])['last_day_close'].diff()
data = data.dropna()
LAST_DAY = data['day'].max()

In [5]:
data.head()

,ticker,close,day,last_day_close,last_day_diff
None,,,,,
2,0,527.00,2,542.0959,-6.9341
3,0,523.90,5,527.0000,-15.0959
4,0,525.31,6,523.9000,-3.1000
5,0,517.10,7,525.3100,1.4100
6,0,523.51,8,517.1000,-8.2100


In [6]:
def ttsplit(df, train_size):
    X = df.drop(['close'], axis = 1)
    y = df['close']
    return train_test_split(X, y, train_size=train_size, random_state=42)

In [7]:
# MODEL 1: A Random Forest Regressor
def build_random_forest(df):
    mean_error = []
    sizes = [1/4]
    for size in sizes:
        xtr, xts, ytr, yts = ttsplit(df, size)

        mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
        mdl.fit(xtr, ytr)

        p = mdl.predict(xts)

        error = mean_squared_error(yts, p)
        print('RMSE Error: %.5f' % (error))
        mean_error.append(error)
    print('Mean Error = %.5f' % np.mean(mean_error))
    return mdl
# forest = build_random_forest(data)

In [8]:
def build_linear_regressor_test(df):
    X_train, X_test, y_train, y_test = ttsplit(df, 0.25)
    
    # Fit and predict
    model = lm.LinearRegression(fit_intercept=True)
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    
    print(f'The validation RMSE for this model is '
          f'{round(mean_squared_error(y_test, y_predicted), 2)}.')

    return model
# linear = build_linear_regressor_test(data)

In [9]:
scaler = pre.StandardScaler()

def build_elastic_net_predictor(df):
    scaler = pre.StandardScaler()
    X_train, X_test, y_train, y_test = ttsplit(df, 0.25)
    
    l1_ratios = np.arange(0, 1.1, .1)
    alphas = np.arange(0.1, 200.1, .1)
    model = lm.ElasticNetCV(l1_ratio=l1_ratios,
                            alphas=alphas,
                            cv=5,
                            fit_intercept=True,
                            max_iter=1000)

    # Fit and predict
    model.fit(scaler.fit_transform(X_train), y_train)
    y_predicted = model.predict(scaler.fit_transform(X_test))

    print(f'The validation RMSE for this model with '
          f'alpha={round(float(model.alpha_), 2)} is '
          f'{round(mean_squared_error(y_test, y_predicted), 2)}.')
    
    return model
# elastic = build_elastic_net_predictor(data)

In [10]:
def predict_future_stock_values(mdl, source_df, days_out):
    abs_day = LAST_DAY + days_out
    # Warning: Columns must be ordered properly for predictor to work!
    tickers = source_df['ticker'].unique()
    days = np.arange(LAST_DAY + 1, abs_day + 1, 1)

    x = source_df.copy()
    for d in days:
        print("DAY {}".format(d))
        
        # Construct a dataframe for the next day, borrowing appropriate values.
        i = x[x['day'] == x['day'].max()]
        i['day'] += 1
        i['last_day_diff'] = i['close'] - i['last_day_close']
        i['last_day_close'] = i['close']
        i = i.drop(['close'], axis=1)
        
        # Predict new close values
        y = mdl.predict(scaler.fit_transform(i))
        i['close'] = pd.Series(y, index=i.index)
        
        x = x.append(i).sort_values(['ticker', 'day'], ascending=[True, True])
        print(x.tail())
#     print(x.head())
#     return mdl.predict(x)
# predict_future_stock_values(elastic, data, 10)

In [43]:
from scipy.optimize import linprog

# Term is in number of days from current!
def suggest_strategy(df, principal, target, term):
#     current_day = df['day'].max()
    current_day = 0
#     elastic = build_elastic_net_predictor(df)
#     extended_data = predict_future_stock_values(elastic, df, term) #extended_data has predictions for future
    extended_data = df
    
    period = extended_data[(extended_data['day'] >= current_day) & (extended_data['day'] <= current_day + term)]
    stocks_with_current_price = period.groupby('ticker').agg({'close': 'first'})
    candidate_stocks = stocks_with_current_price[stocks_with_current_price['close'] <= 2 * principal].index.values
    candidate_stock_data = period.loc[period['ticker'].isin(candidate_stocks)]
    
    c = []
    A_ub = np.array([np.array(stocks_with_current_price.loc[candidate_stocks]['close'])])
    b_ub = np.array([principal])
    for cs in candidate_stocks:
        max_close = candidate_stock_data[candidate_stock_data['ticker'] == cs]['close'].max()
        c.append(max_close)
    
    c = np.array(c)
    optimize_result = linprog(-c, A_ub, b_ub)
    
    print(optimize_result)
    print(c.dot(optimize_result.x))

suggest_strategy(data, 50, 100, 365)

     fun: -100.76495132127955
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([ 0.])
  status: 0
 success: True
       x: array([ 1.73852573,  0.        ])
100.764951321


In [12]:
"""
Improvements:
    - Recurrent Neural Network (e.g. LSTM)
    - Convert time series to stationary
    - Uniform Scaling for each time series
"""

'\nImprovements:\n    - Recurrent Neural Network (e.g. LSTM)\n    - Convert time series to stationary\n    - Uniform Scaling for each time series\n'